### JHESORLEY M. LAID
### MIT-2

In [1]:
import numpy as np
import hashlib
import plotly.graph_objects as go

In [2]:
stream_data = np.random.choice(
    ["A", "B", "C", "D", "E"],
    size=500,
    p=[0.4, 0.25, 0.15, 0.1, 0.1]
)

In [3]:
width = 20
depth = 5

cms = np.zeros((depth, width), dtype=int)

In [4]:
def cms_hash(item, seed):
    return int(hashlib.md5((str(seed) + item).encode()).hexdigest(), 16) % width

In [5]:
def cms_update(item):
    for i in range(depth):
        cms[i, cms_hash(item, i)] += 1

In [6]:
for item in stream_data:
    cms_update(item)

In [7]:
def cms_query(item):
    return min(cms[i, cms_hash(item, i)] for i in range(depth))

exact_counts = {x: list(stream_data).count(x) for x in set(stream_data)}

for k in exact_counts:
    print(f"{k}: Exact={exact_counts[k]}, CMS Estimate={cms_query(k)}")

B: Exact=124, CMS Estimate=124
D: Exact=53, CMS Estimate=53
A: Exact=214, CMS Estimate=214
E: Exact=48, CMS Estimate=48
C: Exact=61, CMS Estimate=61


In [8]:
#Bloom Filter

In [9]:
bf_size = 50
bf = np.zeros(bf_size, dtype=int)
num_hashes = 3

In [10]:
def bf_hash(item, seed):
    return int(hashlib.sha1((str(seed) + item).encode()).hexdigest(), 16) % bf_size

In [11]:
def bf_add(item):
    for i in range(num_hashes):
        bf[bf_hash(item, i)] = 1

for item in stream_data:
    bf_add(item)

In [12]:
def bf_check(item):
    return all(bf[bf_hash(item, i)] == 1 for i in range(num_hashes))

print("A exists?", bf_check("A"))
print("Z exists?", bf_check("Z"))
print("B exists?", bf_check("B"))

A exists? True
Z exists? False
B exists? True


In [13]:
#Hyperloglog

In [14]:
num_registers = 16
registers = np.zeros(num_registers)

In [15]:
def hll_hash(item):
    return int(hashlib.sha1(item.encode()).hexdigest(), 16)

def hll_update(item):
    h = hll_hash(item)
    register = h % num_registers
    leading_zeros = len(bin(h)) - len(bin(h).lstrip("0b1"))
    registers[register] = max(registers[register], leading_zeros)

In [16]:
for item in stream_data:
    hll_update(item)

In [17]:
alpha = 0.7213 / (1 + 1.079 / num_registers)
estimate = alpha * num_registers**2 / np.sum(2 ** -registers)

print("Exact distinct count:", len(set(stream_data)))
print("HyperLogLog estimate:", int(estimate))

Exact distinct count: 5
HyperLogLog estimate: 14


In [18]:
#Matrix Sketching

In [19]:
stream_matrix = np.random.randn(100, 5)
sketch_size = 3
sketch = np.zeros((sketch_size, 5))

In [20]:
def update_matrix_sketch(row):
    global sketch
    sketch = np.vstack([sketch, row])
    if sketch.shape[0] > sketch_size:
        _, _, vt = np.linalg.svd(sketch, full_matrices=False)
        sketch = vt[:sketch_size]

In [21]:
print("Original Matrix Shape:", stream_matrix.shape)
print("Sketched Matrix Shape:", sketch.shape)
print("\nSketched Matrix:\n", np.round(sketch, 3))

Original Matrix Shape: (100, 5)
Sketched Matrix Shape: (3, 5)

Sketched Matrix:
 [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
